In [2]:
from __future__ import division, print_function

In [130]:
import numpy as np
import pandas as pd
import astroquery
import matplotlib.pyplot as plt
import glob

import matplotlib

%matplotlib inline

In [131]:
fns = glob.glob('/Users/tom/Dropbox/TIC4/CTL/*.csv')

header = [
    'RA','DEC','TESSMAG','TEFF','PRIORITY','RADIUS','MASS','CONTRATIO',
    'ECLONG','ECLAT','GLONG','GLAT','PLX','V','J','H','Ks','HIP','TYCHO2',
    '2MASS','TICID','CDWARF','PRIO_NON_CON','TEFF_SRC','RAD_SRC','MASS_SRC'
    ]


In [132]:
# give me the 100000th highest ranked target in each file
# the lowest value will be our minimum priority
minpriority = 1.0
for fn in fns:
    df = pd.read_csv(fn, names=header)
    print(df.sort_values('PRIORITY').iloc[-100000].PRIORITY)
    if df.sort_values('PRIORITY').iloc[-100000].PRIORITY < minpriority:
        minpriority = df.sort_values('PRIORITY').iloc[-100000].PRIORITY

0.00711750080922
0.00765143330282
0.00683916628325
0.00571572081392
0.00463359697578
0.0057790754149
0.00587551323589
0.00519270151783
0.00577585002683
0.00541358617543
0.00438532556096
0.00649876202036


In [223]:
cutvalue = minpriority
df = pd.DataFrame(columns=header)
for fn in fns:
    iter_csv = pd.read_csv(fn, names=header, iterator=True, chunksize=1000)
    df1 = pd.concat([chunk[chunk['PRIORITY'] > cutvalue] for chunk in iter_csv])
    df = pd.concat([df,df1], ignore_index=True) # need to recount the index, or we have duplicates
    print('done{}'.format(fn))
    

done/Users/tom/Dropbox/TIC4/CTL/00-02.csv
done/Users/tom/Dropbox/TIC4/CTL/02-04.csv
done/Users/tom/Dropbox/TIC4/CTL/04-06.csv
done/Users/tom/Dropbox/TIC4/CTL/06-08.csv
done/Users/tom/Dropbox/TIC4/CTL/08-10.csv
done/Users/tom/Dropbox/TIC4/CTL/10-12.csv
done/Users/tom/Dropbox/TIC4/CTL/12-14.csv
done/Users/tom/Dropbox/TIC4/CTL/14-16.csv
done/Users/tom/Dropbox/TIC4/CTL/16-18.csv
done/Users/tom/Dropbox/TIC4/CTL/18-20.csv
done/Users/tom/Dropbox/TIC4/CTL/20-22.csv
done/Users/tom/Dropbox/TIC4/CTL/22-24.csv


In [226]:
# now cut the sky into 26 segments and select top (400,000 / 26) from each
# the cuts are in ecliptic longitute
selectnum = int(np.round(400000/26., ))
df.loc[:,'SELECTED'] = np.zeros(df.shape[0], dtype='bool')

eloop = np.linspace(0,360,14)
espace = np.diff(eloop)[0] # get space in degrees
for elon in eloop[:-1]:
    # southern hemisphere
    ms = (df.ECLONG >= elon) & (df.ECLONG < elon+espace) & (df.ECLAT <= -6)
    selectthese = df.loc[ms].sort_values('PRIORITY').iloc[-selectnum:].index
    df.loc[df.index.isin(selectthese), 'SELECTED'] = True
    
#     # northern hemisphere
    mn = (df.ECLONG >= elon) & (df.ECLONG < elon+espace) & (df.ECLAT >= 6)
    selectthese = df.loc[mn].sort_values('PRIORITY').iloc[-selectnum:].index
    df.loc[df.index.isin(selectthese), 'SELECTED'] = True
    
#     print(df.loc[df.SELECTED == True].shape[0])
    print(elon,elon+espace,df.loc[df.SELECTED == True].shape[0])
    


0.0 27.6923076923 30770
27.6923076923 55.3846153846 61540
55.3846153846 83.0769230769 92310
83.0769230769 110.769230769 123080
110.769230769 138.461538462 153850
138.461538462 166.153846154 184620
166.153846154 193.846153846 215390
193.846153846 221.538461538 246160
221.538461538 249.230769231 276930
249.230769231 276.923076923 307700
276.923076923 304.615384615 338470
304.615384615 332.307692308 369240
332.307692308 360.0 400010


In [228]:
selected = df[df.SELECTED == True]

In [235]:
selected.to_csv('selected.csv.gzip',compression='gzip')